# ERCOT API Example with Authentication

This notebook demonstrates how to use the Tiny Grid SDK to access ERCOT grid data with authentication.

## Prerequisites

1. Register for ERCOT API access at [apiexplorer.ercot.com](https://apiexplorer.ercot.com/)
2. Subscribe to API products to get your subscription key
3. Set up your `.env` file with your credentials (see `.env.example`)

## Setup


In [ ]:
import os
from pathlib import Path

from dotenv import load_dotenv

# Load environment variables from .env file
# Try to find .env in the examples directory or current directory
examples_dir = Path(__file__).parent if '__file__' in globals() else Path.cwd()
env_file = examples_dir / ".env"

# Also try parent directory (in case running from project root)
if not env_file.exists():
    env_file = examples_dir.parent / ".env"

load_dotenv(env_file)

# Verify environment variables are loaded
required_vars = ["ERCOT_USERNAME", "ERCOT_PASSWORD", "ERCOT_SUBSCRIPTION_KEY"]
missing_vars = [var for var in required_vars if not os.getenv(var)]

if missing_vars:
    raise ValueError(
        f"Missing required environment variables: {', '.join(missing_vars)}. "
        f"Please create a .env file at {env_file} with your ERCOT credentials. "
        "See .env.example for the required format."
    )

print("✓ Environment variables loaded successfully")
print(f"✓ Loaded from: {env_file if env_file.exists() else 'environment'}")


: 

## Initialize Authentication


In [ ]:
from tinygrid import ERCOT, ERCOTAuth, ERCOTAuthConfig

# Create authentication configuration from environment variables
auth_config = ERCOTAuthConfig(
    username=os.getenv("ERCOT_USERNAME"),
    password=os.getenv("ERCOT_PASSWORD"),
    subscription_key=os.getenv("ERCOT_SUBSCRIPTION_KEY"),
)

# Create authentication handler
auth = ERCOTAuth(auth_config)

print("✓ Authentication configured")


## Create ERCOT Client with Authentication


In [ ]:
# Create ERCOT client with authentication
ercot = ERCOT(auth=auth)

print(f"✓ ERCOT client created: {ercot}")


## Example 1: Get Load Forecast by Weather Zone


In [ ]:
# Get load forecast for the next 7 days
from datetime import datetime, timedelta

end_date = datetime.now()
start_date = end_date - timedelta(days=7)

forecast = ercot.get_load_forecast_by_weather_zone(
    start_date=start_date.strftime("%Y-%m-%d"),
    end_date=end_date.strftime("%Y-%m-%d"),
    model="WEATHERZONE",
)

print("✓ Retrieved forecast data")
print(f"Number of records: {len(forecast) if isinstance(forecast, dict) else 'N/A'}")

# Display sample data
if isinstance(forecast, dict) and forecast:
    print("\nSample data:")
    for key, value in list(forecast.items())[:5]:
        print(f"  {key}: {value}")


## Example 2: Error Handling


In [ ]:
from tinygrid import GridAPIError, GridAuthenticationError, GridTimeoutError

try:
    # This will use the authenticated client
    data = ercot.get_load_forecast_by_weather_zone(
        start_date="2024-01-01",
        end_date="2024-01-07",
    )
    print("✓ Request successful")
except GridAuthenticationError as e:
    print(f"Authentication error: {e.message}")
    print(f"Status code: {e.status_code}")
except GridAPIError as e:
    print(f"API error: {e.message}")
    print(f"Status code: {e.status_code}")
except GridTimeoutError as e:
    print(f"Timeout error: {e.message}")
    print(f"Timeout: {e.timeout} seconds")


## Notes

- Tokens are automatically cached and refreshed when expired (tokens are valid for 1 hour)
- The subscription key is automatically included in all authenticated requests
- Never commit your `.env` file to version control
- Keep your credentials secure and rotate them regularly
